In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
import json
import matplotlib.pyplot as plt
import os
import tqdm
import pickle
from pathlib import Path
from torch.utils.data import DataLoader
from sklearn.metrics import ndcg_score
np.random.seed(0)

In [ ]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import FreqDist
import re
from nltk.tokenize import RegexpTokenizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag

In [ ]:
!wget https://jmcauley.ucsd.edu/data/amazon_v2/categoryFiles/Cell_Phones_and_Accessories.json.gz --no-check-certificate 

--2023-01-18 13:19:57--  https://jmcauley.ucsd.edu/data/amazon_v2/categoryFiles/Cell_Phones_and_Accessories.json.gz
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 1232323281 (1.1G) [application/x-gzip]
Saving to: ‘Cell_Phones_and_Accessories.json.gz’

Cell_Phones_and_Acc 100%[===================>]   1.15G  17.4MB/s    in 69s     

2023-01-18 13:21:07 (17.0 MB/s) - ‘Cell_Phones_and_Accessories.json.gz’ saved [1232323281/1232323281]



In [ ]:
!gunzip Cell_Phones_and_Accessories.json.gz

In [ ]:
!wget https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Cell_Phones_and_Accessories.json.gz --no-check-certificate 

--2023-01-18 13:21:42--  https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Cell_Phones_and_Accessories.json.gz
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 360006372 (343M) [application/x-gzip]
Saving to: ‘meta_Cell_Phones_and_Accessories.json.gz’

meta_Cell_Phones_an 100%[===================>] 343.33M  17.6MB/s    in 21s     

2023-01-18 13:22:04 (16.4 MB/s) - ‘meta_Cell_Phones_and_Accessories.json.gz’ saved [360006372/360006372]



In [ ]:
save_path='/content/drive/MyDrive/cell_phone/'

In [ ]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    name=b'"verified": \"true\",'
    l=l.replace(b'"verified": true,',bytes(name))
    name1=b'"verified": \"false\",'
    l=l.replace(b'"verified": false,',bytes(name))
    yield eval(l)
def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    rem_list={'reviewerName','verified','reviewTime','summary','reviewText','style','vote','image','brand','details','imageURL','imageURLHighRes','category','date','fit','main_cat','tech1','tech2','also_buy','also_view','similar_item'}
    [d.pop(key) for key in rem_list if key in d.keys()]
    # print(d)
    if i%100000==0:
      print(i)
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
df_meta = getDF('meta_CDs_and_Vinyl.json.gz')

In [ ]:
df_meta

In [ ]:
df_review = getDF('/content/CDs_and_Vinyl.json.gz')

In [ ]:
df_meta.description= df_meta.description.apply(lambda y: np.nan if len(y)==0 else y)

In [ ]:
# df_meta.feature= df_meta.feature.apply(lambda y: np.nan if len(y)==0 else y)

In [ ]:
df_meta=df_meta[~df_meta['description'].isna() & ~df_meta['feature'].isna()]

In [ ]:
df_review.drop(df_review.index[~df_review['asin'].isin(df_meta['asin'])], inplace=True) 

In [ ]:
df_review=df_review[df_review['reviewerID'].isin(df_review['reviewerID'].value_counts()[df_review['reviewerID'].value_counts()>=10].index)]
df_review=df_review[df_review['asin'].isin(df_review['asin'].value_counts()[df_review['asin'].value_counts()>=5].index)]

In [ ]:
df_review

,reviewerID,asin,overall,unixReviewTime
57,AV3DKV2TW5B3C,0005164885,5.0,1515542400
83,A1TBCT7195A7IU,0005164885,5.0,1501718400
117,A30M3WWF54M74L,0005164885,5.0,1483920000
135,A2OE456EJT10IH,0005164885,5.0,1482364800
150,A3LEN0P07MGJE2,0005164885,5.0,1480723200
...,...,...,...,...
4543270,A168SXKC6AJR2V,B01HIDSULW,3.0,1484179200
4543299,A19MT5PNR9BSUN,B01HINXMPG,4.0,1484352000
4543303,A1S7Z0LWRUI67G,B01HINXMPG,5.0,1472169600
4543356,APXJ8QC63EG38,B01HJG3VZI,3.0,1532476800


In [ ]:
df_meta.drop(df_meta.index[~df_meta['asin'].isin(df_review['asin'])], inplace=True)

In [ ]:
df_meta=df_meta.drop(['price'],axis=1)

In [ ]:
df_meta

,description,title,feature,rank,asin
5,"[This is a concept album all the way, with tal...",Christmas Eve and Other Stories,[],73 in CDs & Vinyl (,0005164885
66,[Editorial Reviews\nProduct Description\nItem ...,The Happiest Baby on the Block,[],"135,199 in Movies & TV (",0972179526
95,[<i><b>Ama</b></i>is the deeply personal yet u...,Ama,[],"929,305 in Books (",0980219663
102,[Guided meditation to train your mind to quiet...,Goodbye Worries,[],"6,934 in CDs & Vinyl (",1450734618
114,[Rock-n-roll maven Pat Benatar performs live i...,Live in New Haven 1983 VHS,[],"510,711 in Movies & TV (",1566054109
...,...,...,...,...,...
516866,[INGRID MICHAELSON - IT DOESN'T HAVE TO MAKE S...,It Doesn't Have To Make Sense,[],"31,182 in CDs & Vinyl (",B01HHGAJJ6
516887,"[To Bobby Rush s innumerable fans, he is an Am...",Porcupine Meat,[],"30,453 in CDs & Vinyl (",B01HIDSULW
516891,[CD],Kinda Don't Care,[],"20,152 in CDs & Vinyl (",B01HIDSTAY
516899,[When he's found himself in intersections of p...,Crossroads,[],"72,981 in CDs & Vinyl (",B01HINXMPG


In [ ]:
df_meta.to_csv(save_path+'df_meta.csv')

In [ ]:
df_review=df_review[df_review['asin'].isin(df_meta['asin'])]

In [ ]:
df_review.to_csv(save_path+'df_reviews.csv')

In [ ]:
df_meta=pd.read_csv(save_path+'df_meta.csv')

In [ ]:
df_review=pd.read_csv(save_path+'df_reviews.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
items_list=[]
users_list=[]
review_features={}
f=open(save_path+'Cell_Phones_and_Accessories')
lines=f.readlines()
i=0
for line in lines:
  if i%10000==0:
    print(i)
  i+=1
  params=line.split('@@')
  user_id=params[0].split('@')[0]
  item_id=params[0].split('@')[1]
  # if item_id in df_meta['asin'].values:
  users_list.append(user_id)
  items_list.append(item_id)

0
10000
20000
30000


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModel
import torch
config = AutoConfig.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
tokenizer_LM = BertTokenizer.from_pretrained("bert-base-uncased")
model_LM = BertForMaskedLM.from_pretrained("bert-base-uncased", output_hidden_states=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
items_list=list(set(items_list))

In [ ]:
print(len(items_list))

4239


In [ ]:
with open('/content/cell-phones_items_list.txt') as fi:
  lines=fi.readlines()

In [ ]:
items_list=[]

In [ ]:
for line in lines:
  item_id=line.replace('\n','')
  items_list.append(item_id)

In [ ]:
len(items_list)

397

In [ ]:
import glob
def get_saved_items_list():
  items_list=[]
  names=glob.glob(save_path+"title_bert/*.json")
  for name in names:
    items_list.append(name.split('/')[-1][0:-5].split('_')[-1])
  return items_list

In [ ]:
saved_items_list=get_saved_items_list()

In [ ]:
len(saved_items_list)

1007

In [ ]:
print(saved_items_list)

['B01BLF3O1E', 'B01A7I9QNQ', 'B011BIWAX4', 'B012CCNGI6', 'B01GEJQTGE', 'B00BV96VDY', 'B01EUSL4CU', 'B00NC7DX6Q', 'B00LWT7AXI', 'B00OZN4LYO', 'B00PNI5R20', 'B00Z7S0OTW', 'B00BKRXEIC', 'B01DBAVFI6', 'B01DUWX9T8', 'B00MAWQFZ0', 'B00NG1HS0K', 'B00F0P2FNU', 'B00YA8308M', 'B01AUR1HF4', 'B00Z7SAGZ4', 'B01BM01J1U', 'B010MVRKTA', 'B004XZHY34', 'B018TGGH4E', 'B010MVJM5K', 'B00U7YKO78', 'B00B7ZF3H2', 'B00IS9TCIQ', 'B014CDHQEI', 'B00JRVLSV8', 'B00MRJQ3US', 'B015L26HFS', 'B00ETOEXFG', 'B00UF5Z9K6', 'B00L2MIOC6', 'B00A9S6DDQ', 'B00NN94UJW', 'B0177NZ59S', 'B00GYD782A', 'B00UFCRO0W', 'B00ASN99KG', 'B016CYJDRI', 'B00SLHWTN0', 'B00GR2UH3A', 'B00DXD6E74', 'B00U6EZC5S', 'B00W4PZG4O', 'B003D1QPHK', 'B00KGSW734', 'B001XXUOQI', 'B015XPU7RC', 'B00GNJHRNU', 'B0092KJ9BU', 'B00SZEFDH8', 'B00V80ZQ5K', 'B00IICJSV4', 'B00TJ3DWIG', 'B00DXGX5RI', 'B00HAYWLE2', 'B0110NIK8O', 'B0040HIKRY', 'B00PINR91M', 'B016Y31YE2', 'B00CMDYV0S', 'B00Q7C6FTA', 'B00F1LTKM2', 'B008HTJMY6', 'B00HQMNS8G', 'B00MNAWM92', 'B00DUYBKM0', 'B00Z

In [ ]:
items_list

In [ ]:
items_list=[item for item in items_list if item not in saved_items_list]

In [ ]:
print(len(items_list))

3232


In [ ]:
import glob
items_list=[]
names=glob.glob(save_path+"features_bert/*.json")
for name in names:
  item_id=name.split('/')[-1][0:-5]
  items_list.append(item_id)

In [ ]:
items_list=[item for item in items_list if item in df_meta['asin'].values]

In [ ]:
print(len(items_list))

941


In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
def lemmatization(text):
    result=''
    wordnet = WordNetLemmatizer()
    for token,tag in pos_tag(text):
        pos=tag[0].lower()
        if pos not in ['a', 'r', 'n', 'v']:
            pos='n'
        # if pos in ['n','a']:   
        result+=wordnet.lemmatize(token,pos)+' '
    return result
def remove_stopwords(text):
    en_stopwords = stopwords.words('english')
    en_stopwords+=['this','made','that','the','those','these','may','could','that','without','iii','with','and','This','That','Those','These','the','The','brbr','of','you']
    result = []
    for token in text:
        if token.lower() not in en_stopwords and len(token)>2:
            result.append(token)
            
    return result
def remove_punct(text):
    tokenizer = RegexpTokenizer(r"\w+")
    lst=tokenizer.tokenize(' '.join(text))
    return lst

def remove_tag(text):
    text=' '.join(text)
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def preprocess(text):
  chars=['&','%','#','@','^','>','<','\n','\\','\t',';','"','/']
  stwords=stopwords.words('english')
  for ch in chars:
    text=text.replace(ch,' ')
  text=" ".join(text.split())
  # text=text.lower()
  text_tokenized=word_tokenize(text)
  cleaned_text= remove_stopwords(text_tokenized)
  cleaned_text= remove_punct(cleaned_text)
  # cleaned_text=lemmatization(cleaned_text)
  cleaned_text=remove_tag(cleaned_text)
  cleaned_text=remove_urls(cleaned_text)
  cleaned_text=''.join([i for i in cleaned_text ])
  # splited=cleaned_text.split(' ')
  cleaned_text=cleaned_text.split(' ')
  new_text=''
  last_word=None
  for i in range(0,len(cleaned_text)):
    word=cleaned_text[i]
    if last_word == word:
      continue
    last_word=word
    if len(word)>1:
      new_text+=word+' '
  # print(cleaned_text)
  return new_text[:-1]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
import re
haRegex = re.compile(r'[a-z]*[A-Z]*\d+[a-z]*[A-Z]*')
def preprocess_text_first(text):
  text=text.replace('</b>','.').replace('<br />','.').replace('<b>','.')
  text=text.replace('..','.').replace('..','.').replace('..','.')
  while '<' in text and '>' in text and text.index('<')<text.index('>'):
    toRemove=text[int(text.index('<')):int(text.index('>'))]+'>'
    text=text.replace(toRemove,' ')
  list_to_replace=['nbsp','\n','mso','gte','xml','false','#',',','!','-','\'','\"','[',']','/','\\n','\\','span','a-size-base','a-color-secondary','input type','header name','value','=','<a href= javascript:void(0) class= ','{','}','class=','header','<a href= javascript:void(0)','<','>','href',')','(',';','quot','&',':','javascript']
  for char in list_to_replace:
    text=text.replace(char,' ')
  for i in range(15):
    text=text.replace('  ',' ')
  # while 'if' in text and 'endif' in text and text.index('if')<text.index('endif'):
  #   # print(int(text.index('if')),int(text.index('endif')))
  #   toRemove=text[int(text.index('if')):int(text.index('endif'))]+'endif'
  #   text=text.replace(toRemove,' ')
  new_text=''
  splited=text.split(' ')
  for i in range(0,len(splited)):
    word=splited[i]
    regFound=haRegex.search(word) 
    # if len(word)>1 and len(word)<35 and regFound == None:
    if len(word)>1 and len(word)<35:
      new_text+=word+' '
  # new_text=lemmatization(new_text)
  # print(new_text)
  return new_text[:-1]

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model=model.to(device)

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_LM=model_LM.to(device)

In [ ]:
print(len(items_list))

941


In [ ]:
def get_replace_words(sent,word):
  words=[]
  sent=sent.lower()
  new_sent=sent.replace(word,'[MASK]')
  inputs = tokenizer_LM(new_sent, return_tensors="pt").to(device)
  mask_token_index = (inputs.input_ids == tokenizer_LM.mask_token_id)[0].nonzero(as_tuple=True)[0]
  with torch.no_grad():
      outputs = model_LM(**inputs)
  # predicted_token_id = outputs.logits[0, mask_token_index].argmax(axis=-1)
  sorted=outputs.logits[0,mask_token_index].sort(axis=-1)
  # print(sorted)
  ids=sorted.indices[0][-5:]
  logits=sorted.values[0][-5:]
  all_logits=[]
  i=0
  for id in ids:
    # print(id)
    new_word=tokenizer.decode(id).replace(' ','')
    words.append(new_word)
    all_logits.append(logits[i].tolist())
    i+=1
  if word not in words:
    ids=sorted.indices[0][-10:]
    for id in ids:
      new_word=tokenizer.decode(id).replace(' ','')
      if word == new_word:
        ind=sorted.indices[0].tolist().index(id)
        all_logits.append(sorted.values[0][ind].tolist())
        words.append(word)
        break
  if word not in words:
    all_logits.append(5.0)
    words.append(word)
  return words,all_logits

In [ ]:
inputs = tokenizer(new_sent, return_tensors="pt").to(device)

In [ ]:
inputs

In [ ]:
def get_replace_vects(sent,word,replaced_words):
  sent=sent.lower()
  vects=[]
  for new_word in replaced_words:
    new_sent=sent.replace(word,new_word)
    inputs = tokenizer(new_sent, return_tensors="pt").to(device)
    if len(inputs.input_ids[0])<513:
      outputs = model(**inputs)
      hiddens=outputs.last_hidden_state
      tokenized=tokenizer.tokenize(new_sent)
      if new_word in tokenized:
        ind=tokenized.index(new_word)
        vects.append(list(hiddens[0][ind+1].detach().cpu().numpy()))
      else:
        vects.append(None)
    else:
      vects.append(None)
  return vects

In [ ]:
k=0  
dict_bert1={'logit':[],'replaced_word':[],'description_words':[],'description_vects':[]}
dict_bert2={'logit':[],'replaced_word':[],'title_words':[],'title_vects':[]}
dict_bert3={'logit':[],'replaced_word':[],'feature_words':[],'feature_vects':[]}
for item_id in items_list:
  if k%1==0:
    print(k)
  k+=1
  # if k<1401:
  #   continue
  # if j==2:
  #   break
  # if j<6270:
  #   continue
  allOk=True
  row=df_meta[df_meta['asin']==item_id]
  if(len(row['description'].values[0])>0):
    description=preprocess_text_first(str(row['description'].values[0]))
    description=list(description.split(','))
    title=preprocess_text_first(row['title'].values[0])
    features= preprocess_text_first(str(row['feature'].values[0]))
    features=list(features.split(','))

    for desc in description:
      
      sents_desc= desc.split('.')
      for sent in sents_desc:
        # if sent[0]==' ':
        #   sent=sent[1:]
        if len(sent.split(' '))>512  or len(sent)<3 or len(sent.split(' '))<2 :
          continue
        if len(sent.split(' '))>400:
          sent=' '.join(list(set(sent.split(' '))))
        new_sent=preprocess(sent).lower()
        # print(sent)
        # print(new_sent)
        tokenized_old=tokenizer.tokenize(sent)
        tokenized_new=tokenizer.tokenize(new_sent)
        if len(tokenized_old)>=512:
          continue
        # print(tokenized_old)
        # print(tokenized_new)
        # print(len(tokenized_old))
        inputs1 = tokenizer(sent, return_tensors="pt").to(device)
        outputs1 = model(**inputs1)
        hiddens1=outputs1.last_hidden_state
        not_in_vocab1=[]
        for word in new_sent.split(' '):
          if word not in tokenizer.vocab.keys():
            not_in_vocab1.append(word)

        for i in range(len(tokenized_old)):
          if tokenized_old[i] != '[UNK]' and tokenized_old[i] in tokenized_new  and tokenized_old[i] in new_sent.split(' '):
            # dict_bert1['description_words'].append(tokenized_old[i])
            # dict_bert1['description_vects'].append(list(hiddens1[0][i+1].detach().cpu().numpy()))
            # dict_bert1['item_id'].append(item_id)
            # print(tokenized_old[i])
            replaced_words,logits=get_replace_words(sent,tokenized_old[i])
            # print(tokenized_old[i],replaced_words,logits)
            vects_replaced_words=get_replace_vects(sent,tokenized_old[i],replaced_words)
            for j in range(len(replaced_words)):
              if vects_replaced_words[j]!=None:
                dict_bert1['description_words'].append(tokenized_old[i])
                dict_bert1['replaced_word'].append(replaced_words[j])
                dict_bert1['logit'].append(logits[j])
                dict_bert1['description_vects'].append(vects_replaced_words[j])
                # dict_bert1['item_id'].append(item_id)
              
        for word in not_in_vocab1:
          tokenized_word=tokenizer.tokenize(word)
          #  replaced_words,logits=get_replace_words(sent,word)
          #  print(word,replaced_words)
          vect1=[]
          #  print(word)
          #  print(len(word))
          for i in range(len(tokenized_old)):
              if tokenized_old[i] in tokenized_word:
                vect1.append(list(hiddens1[0][i+1].detach().cpu().numpy()))
          if len(vect1)>0:
            dict_bert1['description_words'].append(word)
            dict_bert1['replaced_word'].append(word)
            dict_bert1['logit'].append(12.0)
            dict_bert1['description_vects'].append(list(np.average(vect1,axis=0))) 
            # dict_bert1['item_id'].append(item_id)

    if len(dict_bert1['description_vects'])==0:
      print(item_id)
      allOk=False
      continue

    new_title=preprocess(title).lower()
    tokenized_old=tokenizer.tokenize(title)
    tokenized_new=tokenizer.tokenize(new_title)
    inputs2 = tokenizer(title, return_tensors="pt").to(device)
    outputs2 = model(**inputs2)
    hiddens2=outputs2.last_hidden_state
    not_in_vocab2=[]
    for word in new_title.split(' '):
      if word not in tokenizer.vocab.keys():
        not_in_vocab2.append(word)
    for i in range(len(tokenized_old)):
      if tokenized_old[i] != '[UNK]' and tokenized_old[i] in tokenized_new and tokenized_old[i] in new_title.split(' '):
        replaced_words,logits=get_replace_words(title,tokenized_old[i])
            # print(tokenized_old[i],replaced_words,logits)
        vects_replaced_words=get_replace_vects(title,tokenized_old[i],replaced_words)
        for j in range(len(replaced_words)):
          if vects_replaced_words[j]!=None:
            dict_bert2['title_words'].append(tokenized_old[i])
            dict_bert2['replaced_word'].append(replaced_words[j])
            dict_bert2['logit'].append(logits[j])
            dict_bert2['title_vects'].append(vects_replaced_words[j])
            # dict_bert2['item_id'].append(item_id)
    for word in not_in_vocab2:
        tokenized_word=tokenizer.tokenize(word)
        vect2=[]
        for i in range(len(tokenized_old)):
          if tokenized_old[i] in tokenized_word:
            vect2.append(list(hiddens2[0][i+1].detach().cpu().numpy()))
        if len(vect2)>0:
          dict_bert2['title_words'].append(word)
          dict_bert2['replaced_word'].append(word)
          dict_bert2['logit'].append(12.0)
          dict_bert2['title_vects'].append(list(np.average(vect2,axis=0))) 
          # dict_bert2['item_id'].append(item_id)

    if len(dict_bert2['title_vects'])==0:
      print(item_id)
      allOk=False
      continue

    for feature in features:
      sents_feature= feature.split('.')
      for sent in sents_feature:
        if len(sent.split(' '))>512  or len(sent)<2 or len(sent.split(' '))<2:
          continue
        if len(sent.split(' '))>300:
          sent=' '.join(list(set(sent.split(' '))))
        new_sent=preprocess(sent).lower()
        tokenized_old=tokenizer.tokenize(sent)
        tokenized_new=tokenizer.tokenize(new_sent)
        # print(sent)
        # print(tokenized_old)
        inputs2 = tokenizer(sent, return_tensors="pt").to(device)
        outputs2 = model(**inputs2)
        hiddens2=outputs2.last_hidden_state
        not_in_vocab3=[]
        for word in new_sent.split(' '):
          if word not in tokenizer.vocab.keys():
            not_in_vocab3.append(word)
        for i in range(len(tokenized_old)):
          if tokenized_old[i] != '[UNK]' and tokenized_old[i] in tokenized_new and tokenized_old[i] in new_sent.split(' '):
            replaced_words,logits=get_replace_words(sent,tokenized_old[i])
            # print(tokenized_old[i],replaced_words,logits)
            vects_replaced_words=get_replace_vects(sent,tokenized_old[i],replaced_words)
            for j in range(len(replaced_words)):
              if vects_replaced_words[j]!=None:
                dict_bert3['feature_words'].append(tokenized_old[i])
                dict_bert3['replaced_word'].append(replaced_words[j])
                dict_bert3['logit'].append(logits[j])
                dict_bert3['feature_vects'].append(vects_replaced_words[j])
                # dict_bert3['item_id'].append(item_id)
        for word in not_in_vocab3:
          tokenized_word=tokenizer.tokenize(word)
          vect3=[]
          for i in range(len(tokenized_old)):
            if tokenized_old[i] in tokenized_word:
              vect3.append(list(hiddens2[0][i+1].detach().cpu().numpy()))
          if len(vect3)>0:
            dict_bert3['feature_words'].append(word)
            dict_bert3['replaced_word'].append(word)
            dict_bert3['logit'].append(12.0)
            dict_bert3['feature_vects'].append(list(np.average(vect3,axis=0)))
            # dict_bert3['item_id'].append(item_id) 
    if len(dict_bert3['feature_vects'])==0:
      print(item_id)
      allOk=False
      continue
    dframe_bert2=pd.DataFrame.from_dict(dict_bert2)
    dframe_bert2.to_json(save_path+'title_bert/'+'df_bert_title_{}.json'.format(item_id))
    dframe_bert3=pd.DataFrame.from_dict(dict_bert3)
    dframe_bert3.to_json(save_path+'feature_bert/'+'df_bert_feature_{}.json'.format(item_id))
    dframe_bert1=pd.DataFrame.from_dict(dict_bert1)
    dframe_bert1.to_json(save_path+'description_bert/'+'df_bert_desc_{}.json'.format(item_id))
    dict_bert1={'logit':[],'replaced_word':[],'description_words':[],'description_vects':[]}
    dict_bert2={'logit':[],'replaced_word':[],'title_words':[],'title_vects':[]}
    dict_bert3={'logit':[],'replaced_word':[],'feature_words':[],'feature_vects':[]}

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
B005SVXCNI
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274

In [ ]:
df_meta[df_meta['asin']==item_id]

,Unnamed: 0,description,title,feature,rank,asin
6784,281087,['B00LUFLG20'],Selfie Stick | Use as GoPro Pole and Monopod |...,['PREMIUM PRIVACY TECHNOLOGY: Content on scree...,"['>#140,536 in Cell Phones & Accessories (See ...",B00LUFLG20


In [ ]:
k=0  
for item_id in items_list:
  dict_bert1={'logit':[],'replaced_word':[],'description_words':[],'description_vects':[]}
  dict_bert2={'logit':[],'replaced_word':[],'title_words':[],'title_vects':[]}
  # dict_bert3={'logit':[],'replaced_word':[],'feature_words':[],'feature_vects':[]}
  if k%10==0:
    print(k)
  k+=1
  # if j==2:
  #   break
  # if j<6270:
  #   continue
  allOk=True
  row=df_meta[df_meta['asin']==item_id]
  description=preprocess_text_first(str(row['description'].values[0]))
  description=list(description.split(','))
  # print(str(row['description'].values[0]))
  title=preprocess_text_first(row['title'].values[0])
  # features= preprocess_text_first(str(row['feature'].values[0]))
  # features=list(features.split(','))
  for desc in description:
    sents_desc= desc.split('.')
    for sent in sents_desc:
      # if sent[0]==' ':
      #   sent=sent[1:]
      if len(sent.split(' '))>512  or len(sent)<3 or len(sent.split(' '))<2 :
        continue
      if len(sent.split(' '))>300:
        sent=' '.join(list(set(sent.split(' '))))
      new_sent=preprocess(sent).lower()
      # print(sent)
      # print(new_sent)
      tokenized_old=tokenizer.tokenize(sent)
      tokenized_new=tokenizer.tokenize(new_sent)
      if len(tokenized_old)>=512:
        continue
      # print(tokenized_old)
      # print(tokenized_new)
      # print(len(tokenized_old))
      inputs1 = tokenizer(sent, return_tensors="pt").to(device)
      outputs1 = model(**inputs1)
      hiddens1=outputs1.last_hidden_state
      not_in_vocab1=[]
      for word in new_sent.split(' '):
        if word not in tokenizer.vocab.keys():
          not_in_vocab1.append(word)

      for i in range(len(tokenized_old)):
        if tokenized_old[i] != '[UNK]' and tokenized_old[i] in tokenized_new  and tokenized_old[i] in new_sent.split(' '):
          # dict_bert1['description_words'].append(tokenized_old[i])
          # dict_bert1['description_vects'].append(list(hiddens1[0][i+1].detach().cpu().numpy()))
          # dict_bert1['item_id'].append(item_id)
          replaced_words,logits=get_replace_words(sent,tokenized_old[i])
          # print(tokenized_old[i],replaced_words,logits)
          vects_replaced_words=get_replace_vects(sent,tokenized_old[i],replaced_words)
          for j in range(len(replaced_words)):
            if vects_replaced_words[j]!=None:
              dict_bert1['description_words'].append(tokenized_old[i])
              dict_bert1['replaced_word'].append(replaced_words[j])
              dict_bert1['logit'].append(logits[j])
              dict_bert1['description_vects'].append(vects_replaced_words[j])
              # dict_bert1['item_id'].append(item_id)
            
      for word in not_in_vocab1:
        tokenized_word=tokenizer.tokenize(word)
        vect1=[]
        #  print(word)
        #  print(len(word))
        for i in range(len(tokenized_old)):
            if tokenized_old[i] in tokenized_word:
              vect1.append(list(hiddens1[0][i+1].detach().cpu().numpy()))
        if len(vect1)>0:
          dict_bert1['description_words'].append(word)
          dict_bert1['replaced_word'].append(word)
          dict_bert1['logit'].append(12.0)
          dict_bert1['description_vects'].append(list(np.average(vect1,axis=0))) 
          # dict_bert1['item_id'].append(item_id)

  if len(dict_bert1['description_vects'])==0:
    print(item_id)
    allOk=False
    continue

  new_title=preprocess(title).lower()
  tokenized_old=tokenizer.tokenize(title)
  tokenized_new=tokenizer.tokenize(new_title)
  inputs2 = tokenizer(title, return_tensors="pt").to(device)
  outputs2 = model(**inputs2)
  hiddens2=outputs2.last_hidden_state
  not_in_vocab2=[]
  for word in new_title.split(' '):
    if word not in tokenizer.vocab.keys():
      not_in_vocab2.append(word)
  for i in range(len(tokenized_old)):
    if tokenized_old[i] != '[UNK]' and tokenized_old[i] in tokenized_new and tokenized_old[i] in new_title.split(' '):
      replaced_words,logits=get_replace_words(title,tokenized_old[i])
          # print(tokenized_old[i],replaced_words,logits)
      vects_replaced_words=get_replace_vects(title,tokenized_old[i],replaced_words)
      for j in range(len(replaced_words)):
        if vects_replaced_words[j]!=None:
          dict_bert2['title_words'].append(tokenized_old[i])
          dict_bert2['replaced_word'].append(replaced_words[j])
          dict_bert2['logit'].append(logits[j])
          dict_bert2['title_vects'].append(vects_replaced_words[j])
          # dict_bert2['item_id'].append(item_id)
  for word in not_in_vocab2:
      tokenized_word=tokenizer.tokenize(word)
      vect2=[]
      for i in range(len(tokenized_old)):
        if tokenized_old[i] in tokenized_word:
          vect2.append(list(hiddens2[0][i+1].detach().cpu().numpy()))
      if len(vect2)>0:
        dict_bert2['title_words'].append(word)
        dict_bert2['replaced_word'].append(word)
        dict_bert2['logit'].append(12.0)
        dict_bert2['title_vects'].append(list(np.average(vect2,axis=0))) 
        # dict_bert2['item_id'].append(item_id)

  if len(dict_bert2['title_vects'])==0:
    print(item_id)
    allOk=False
    continue

  # for feature in features:
  #   sents_feature= feature.split('.')
  #   for sent in sents_feature:
  #     if len(sent.split(' '))>512  or len(sent)<2 or len(sent.split(' '))<2:
  #       continue
  #     if len(sent.split(' '))>300:
  #       sent=' '.join(list(set(sent.split(' '))))
  #     new_sent=preprocess(sent).lower()
  #     tokenized_old=tokenizer.tokenize(sent)
  #     tokenized_new=tokenizer.tokenize(new_sent)
  #     # print(sent)
  #     # print(tokenized_old)
  #     inputs2 = tokenizer(sent, return_tensors="pt").to(device)
  #     outputs2 = model(**inputs2)
  #     hiddens2=outputs2.last_hidden_state
  #     not_in_vocab3=[]
  #     for word in new_sent.split(' '):
  #       if word not in tokenizer.vocab.keys():
  #         not_in_vocab3.append(word)
  #     for i in range(len(tokenized_old)):
  #       if tokenized_old[i] != '[UNK]' and tokenized_old[i] in tokenized_new and tokenized_old[i] in new_sent.split(' '):
  #         replaced_words,logits=get_replace_words(sent,tokenized_old[i])
  #         # print(tokenized_old[i],replaced_words,logits)
  #         vects_replaced_words=get_replace_vects(sent,tokenized_old[i],replaced_words)
  #         for j in range(len(replaced_words)):
  #           if vects_replaced_words[j]!=None:
  #             dict_bert3['feature_words'].append(tokenized_old[i])
  #             dict_bert3['replaced_word'].append(replaced_words[j])
  #             dict_bert3['logit'].append(logits[j])
  #             dict_bert3['feature_vects'].append(vects_replaced_words[j])
  #     for word in not_in_vocab3:
  #       tokenized_word=tokenizer.tokenize(word)
  #       vect3=[]
  #       for i in range(len(tokenized_old)):
  #         if tokenized_old[i] in tokenized_word:
  #           vect3.append(list(hiddens2[0][i+1].detach().cpu().numpy()))
  #       if len(vect3)>0:
  #         dict_bert3['feature_words'].append(word)
  #         dict_bert3['replaced_word'].append(word)
  #         dict_bert3['logit'].append(12.0)
  #         dict_bert3['feature_vects'].append(list(np.average(vect3,axis=0)))
  # if len(dict_bert3['feature_vects'])==0:
  #   print(item_id)
  #   allOk=False
  #   continue
  dframe_bert2=pd.DataFrame.from_dict(dict_bert2)
  dframe_bert2.to_json(save_path+'titles_bert/'+'{}.json'.format(item_id))
  # dframe_bert3=pd.DataFrame.from_dict(dict_bert3)
  # dframe_bert3.to_json(save_path+'features_bert/'+'{}.json'.format(item_id))
  dframe_bert1=pd.DataFrame.from_dict(dict_bert1)
  dframe_bert1.to_json(save_path+'descriptions_bert/'+'{}.json'.format(item_id))

0
B0015UGNSC
B00F05P8J8
B003IMV8IA
B0000ALFZR
B0000004VW
10
B008OIQYWS
B00006L3JC
B00006JJ6L
B000008U7J
B00008NG5V
20
B00004SPQK
B0000A4GHX
B000A7Q1P6
30
B000002VTC
B0007GAEGC
B0000029FR
B004WKOCQC
40
B00000DRA8
B000001GYM
B000283OAI
B00ISYLBHG
50
B00097DXZS
B000002ON9
B000001ELH
60
B00005LB1H
B00004OCEO
B000UGG39Q
70
B0000029GI
B001QITO1Q
B00005066D
B0000026BH
80
B0012GMZ4K
90
B000068GST
B00000JWN0
B0002NRRYM
100
B000NVL4EW
B0012GN32I
B0000028R8
110
B0000024JA
B00004VW2P
B00000E6BV
B00454U1O2
120
B000024JEX
B000055XWR
B0000025AS
130
B00LGQVDRG
B00005ASLC
B000002J5X
B0000032N0
B000FNO1DO
140
B013LTUKNI
B00IWS73PW
B00HZMRXZG
B000002J2D
B00FA29FDQ
150
B000002VEE
B0052FPPTY
160
B000002VR3
B004ZN8MH6
B005HWUBZ0
B000006TKC
B015HE05O4
170
B0057VDGNK
B000063CNL
B004DWKO8Y
B000002B3Q
B000002UAM
180
B0015I0HMC
B000006SUJ
190
B000002GB7
B000068W4A
B000002IJA
200
B000044U46
B00029P9X2
B002AOWXQ8
B000006X37
B000008DV6
B000005RYN
B00000IQMI
210
B006ZC57FA
B00FNRA7RQ
B00006IX86
220
B0000CC6QD
B000RG